In [8]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import numpy as np
import sys
import time
import cv2
from Levenshtein import distance as levenshtein_distance
import textdistance

Pasii pentru autentificare cu computer vision:

In [9]:
'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "Flav9rWsscbMVxrkg9RWPzof9OmsEPHhdalJqHm1TmJ6177zAbFHJQQJ99BCACi5YpzXJ3w3AAAFACOG3wdH"
endpoint = "https://teodorvitelaru.cognitiveservices.azure.com/"
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''

'\nEND - Authenticate\n'

In [45]:
# 2. Funcție pentru recunoașterea textului scris de mână
def recognize_handwritten_text(image_path):
    with open(image_path, "rb") as image:
        # Trimite imaginea pentru procesare
        read_response = computervision_client.read_in_stream(image, raw=True)
        read_operation_location = read_response.headers["Operation-Location"]
        operation_id = read_operation_location.split("/")[-1]

        # Așteaptă finalizarea procesului
        while True:
            read_result = computervision_client.get_read_result(operation_id)
            if read_result.status not in ["notStarted", "running"]:
                break
            time.sleep(1)

        # Extrage textul recunoscut
        if read_result.status == OperationStatusCodes.succeeded:
            extracted_text = []
            for page in read_result.analyze_result.read_results:
                for line in page.lines:
                    extracted_text.append({
                        "text": line.text,
                        "bounding_box": line.bounding_box
                    })
            return extracted_text
        else:
            return None

# 3. Funcție pentru desenarea chenarelor roșii pe imagine
def draw_text_boxes(image_path, recognized_text, output_path):
    # Citește imaginea folosind OpenCV
    image = cv2.imread(image_path)
    original_image = cv2.imread(image_path)

    for line in recognized_text:
        # Coordonatele locației textului (bounding box)
        bounding_box = line["bounding_box"]
        x1, y1 = int(bounding_box[0]), int(bounding_box[1])
        x3, y3 = int(bounding_box[4]), int(bounding_box[5])

        # Desenează un chenar roșu în jurul textului
        cv2.rectangle(image, (x1, y1), (x3, y3), (0, 0, 255), 2)

        # Adaugă textul recunoscut deasupra chenarului
        cv2.putText(image, line["text"], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    # Salvează imaginea procesată
    cv2.imwrite(output_path, image)
    print(f"Imaginea procesată a fost salvată la: {output_path}")


1.Detectarea textului din imagini si punerea in border box rosu a
textului identificat. Output ul imaginii "i" este in "./output images/outout{i}_image.jpg"


In [46]:

for i in range(1, 6):
    image_path = f"test{i}.png"  # Înlocuiește cu calea către imaginea ta
    recognized_text = recognize_handwritten_text(image_path)
    if recognized_text:
        print("Text recunoscut:")
        for line in recognized_text:
            print(f"Text: {line['text']}, Bounding Box: {line['bounding_box']}")

        # Desenare chenare și afișare imagine
        draw_text_boxes(image_path, recognized_text, f"./output images/output{i}_image.png")
    
    else:
        print("Nu s-a putut recunoaște textul.")

Text recunoscut:
Text: Google Cloud, Bounding Box: [174.0, 53.0, 417.0, 51.0, 418.0, 93.0, 174.0, 97.0]
Text: Platform, Bounding Box: [235.0, 112.0, 349.0, 110.0, 350.0, 149.0, 235.0, 151.0]
Imaginea procesată a fost salvată la: ./output images/output1_image.png
Text recunoscut:
Text: Lucces in resolvarea, Bounding Box: [86.0, 314.0, 1335.0, 287.0, 1336.0, 443.0, 86.0, 478.0]
Text: TEMELOR la, Bounding Box: [140.0, 590.0, 1045.0, 587.0, 1046.0, 723.0, 140.0, 727.0]
Text: LABORA toarele de, Bounding Box: [81.0, 915.0, 1007.0, 926.0, 1004.0, 1039.0, 78.0, 1014.0]
Text: Inteligenta Artificialà!, Bounding Box: [108.0, 1129.0, 1450.0, 1151.0, 1446.0, 1293.0, 105.0, 1259.0]
Imaginea procesată a fost salvată la: ./output images/output2_image.png
Text recunoscut:
Text: Teodor, Bounding Box: [354.0, 102.0, 616.0, 134.0, 605.0, 237.0, 345.0, 212.0]
Text: cânta, Bounding Box: [376.0, 263.0, 570.0, 278.0, 566.0, 366.0, 367.0, 354.0]
Imaginea procesată a fost salvată la: ./output images/output3_ima

Afisarea erorii pentru cuvinte si litere gresite.

In [8]:

# Text corect (ground truth)
ground_truth = [
    "Succes in rezolvarea",
    "TEMELOR la",
    "LABORATOARE de",
    "Inteligenta Artificiala!"
]

ground_truth2 = [
    "Google Cloud",
    "Platform",
]


# Funcție pentru calculul CER (Character Error Rate)
def cer(gt, ocr):
    total_chars = sum(len(line) for line in gt)
    errors = sum(levenshtein_distance(gt_line, ocr_line) for gt_line, ocr_line in zip(gt, ocr))
    return errors / total_chars if total_chars > 0 else 0

# Funcție pentru calculul WER (Word Error Rate)
def wer(gt, ocr):
    gt_words = " ".join(gt).split()
    ocr_words = " ".join(ocr).split()
    return levenshtein_distance(" ".join(gt_words), " ".join(ocr_words)) / len(gt_words) if len(gt_words) > 0 else 0

# Funcție pentru calculul altor metrici de distanță
def calculate_distances(gt, ocr):
    results = []
    for i, (gt_line, ocr_line) in enumerate(zip(gt, ocr)):
        levenshtein = levenshtein_distance(gt_line, ocr_line)

        # Calcul Hamming: Trunchiere la lungimea minimă
        min_length = min(len(gt_line), len(ocr_line))
        truncated_gt = gt_line[:min_length]
        truncated_ocr = ocr_line[:min_length]
        hamming = textdistance.hamming.normalized_similarity(truncated_gt, truncated_ocr) if min_length > 0 else "N/A"

        results.append({
            "line_number": i + 1,
            "ground_truth": gt_line,
            "recognized_text": ocr_line,
            "levenshtein": levenshtein,
            "hamming": hamming,
        })
    return results

image_path = "test2.jpeg" 

# Recunoaștere text
recognized_text = recognize_handwritten_text(image_path)
if recognized_text:
    # Extrage textul recunoscut pe linii
    result = [line["text"] for line in recognized_text]

    # Afișează textul recunoscut
    print("Text recunoscut:")
    for line in result:
        print(line)

    # Calcul metrici
    cer_score = cer(ground_truth, result)
    wer_score = wer(ground_truth, result)
    distances = calculate_distances(ground_truth, result)

    print(f"\nCER (Character Error Rate): {cer_score:.2%}")
    print(f"WER (Word Error Rate): {wer_score:.2%}")

    print("\nMetrici de distanță pentru fiecare linie:")
    for dist in distances:
        print(f"Linia {dist['line_number']}:")
        print(f"  Ground Truth: {dist['ground_truth']}")
        print(f"  Recunoscut: {dist['recognized_text']}")
        print(f"  Levenshtein: {dist['levenshtein']}")
        print(f"  Hamming: {dist['hamming']}")

else:
    print("Nu s-a putut recunoaște textul.")

Text recunoscut:
Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!

CER (Character Error Rate): 16.18%
WER (Word Error Rate): 122.22%

Metrici de distanță pentru fiecare linie:
Linia 1:
  Ground Truth: Succes in rezolvarea
  Recunoscut: Lucces in resolvarea
  Levenshtein: 2
  Hamming: 0.9
Linia 2:
  Ground Truth: TEMELOR la
  Recunoscut: TEMELOR la
  Levenshtein: 0
  Hamming: 1.0
Linia 3:
  Ground Truth: LABORATOARE de
  Recunoscut: LABORA toarele de
  Levenshtein: 8
  Hamming: 0.5
Linia 4:
  Ground Truth: Inteligenta Artificiala!
  Recunoscut: Inteligenta Artificialà!
  Levenshtein: 1
  Hamming: 0.9583333333333334


Desi am dat un text aruncat, a fost capabil sa l identifice corect

Functiile pentru calcularea coordonatelor pentru border box, respectiv functia
care calculeaza intersectia celor doua imagini:

In [ ]:
def extract_bounding_box_from_image(image_path):
    # Citim imaginea
    image = cv2.imread(image_path)
    if image is None:
        return None

    # Convertim imaginea în spațiul de culoare HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

   # Definim intervalul pentru culoarea roșie
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])


    # Creăm o mască pentru culoarea roșie
    mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    red_mask = mask1 | mask2

    # Detectăm contururile
    contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Găsim bounding box-ul pentru cel mai mare contur (dacă există)
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, x + w, y + h))
    
    # Dacă nu s-au găsit contururi, returnăm None
    if not bounding_boxes:
        return None
    
    return bounding_boxes

def calculate_iou(boxesA, boxesB):
    iou_scores = []

    # Iterăm prin perechile corespunzătoare
    for boxA, boxB in zip(boxesA, boxesB):
        print(f"Box A: {boxA}, Box B: {boxB}")
        # Calculăm coordonatele intersecției
        xA = max(boxA[0], boxB[0])  # x_min
        yA = max(boxA[1], boxB[1])  # y_min
        xB = min(boxA[2], boxB[2])  # x_max
        yB = min(boxA[3], boxB[3])  # y_max

        # Calculăm aria intersecției
        inter_width = max(0, xB - xA)
        inter_height = max(0, yB - yA)
        inter_area = inter_width * inter_height

        # Calculăm ariile bounding box-urilor
        boxA_area = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        boxB_area = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

        # Calculăm aria uniunii
        union_area = boxA_area + boxB_area - inter_area

        # Calculăm IoU
        iou = inter_area / union_area if union_area > 0 else 0
        iou_scores.append(iou)

    return iou_scores

Calcularea corectitudinii identificarii textului:

In [ ]:
def calculate_localization_quality():
    for i in range(1, 6): 
        # Construim numele fișierelor
        original_image_path = f"./images boxBorder/test{i}_border.png"
        border_image_path = f"./output images/output{i}_image.png"

        # Extragem bounding box-ul din imaginea originală
        original_box = extract_bounding_box_from_image(original_image_path)
        if original_box is None:
            print(f"Nu s-a găsit un bounding box în imaginea {original_image_path}.")
            continue
        predict_box = extract_bounding_box_from_image(border_image_path)
        if predict_box is None:
            print(f"Nu s-a găsit un bounding box în imaginea {border_image_path}.")
            continue
        # Calculăm IoU
        iou = calculate_iou(original_box, predict_box)
        print(f"Calitate localizare pentru imaginea {i}: {iou}")

calculate_localization_quality()

Box A: (235, 112, 353, 156), Box B: (234, 111, 352, 151)
Box A: (177, 40, 419, 106), Box B: (173, 52, 420, 103)
Calitate localizare pentru imaginea 1: [0.8530566461020751, 0.760584211499353]
Box A: (109, 1106, 1455, 1374), Box B: (107, 1128, 1448, 1295)
Box A: (78, 908, 1006, 1028), Box B: (278, 1118, 280, 1120)
Box A: (133, 576, 1048, 725), Box B: (271, 1118, 273, 1120)
Box A: (71, 294, 1336, 462), Box B: (262, 1118, 264, 1120)
Calitate localizare pentru imaginea 2: [0.619320227550947, 0.0, 0.0, 0.0]
Box A: (373, 263, 571, 365), Box B: (375, 262, 568, 368)
Box A: (323, 100, 625, 237), Box B: (398, 252, 400, 254)
Calitate localizare pentru imaginea 3: [0.9388592140404426, 0.0]
Box A: (374, 239, 734, 353), Box B: (370, 243, 718, 340)
Box A: (311, 77, 751, 215), Box B: (425, 227, 437, 235)
Calitate localizare pentru imaginea 4: [0.805445592352998, 0.0]
Box A: (247, 28, 1078, 286), Box B: (302, 48, 906, 169)
Calitate localizare pentru imaginea 5: [0.3408800455228127]


In [12]:
total = [
    [
    "Google Cloud",
    "Platform",
],
    [
    "Succes in rezolvarea",
    "TEMELOR la",
    "LABORATOARE de",
    "Inteligenta Artificiala!"
],
    [
        "Teodor",
        "canta"
    ],
    [
        "Hai sa ne",
        "intalnim!"
    ],
    [
        "Acesta este",
        "un scris urat"
    ],
    [
        "Daca a=1, atunci",
        "7<Ana sau 8>9",
        "7-5=2"
    ]

]

Se poate observa ca daca scrisul este mai urat, cu atat mai greu devine intelegerea lui.
De asemenea, simbolurile nu sunt identificate chiar complet.

In [ ]:

for i in range(1, 7):
    image_path = f"test{i}.png"
    ground_truth = total[i-1]
    print(f"Text: {ground_truth}")
    recognized_text = recognize_handwritten_text(image_path)
    if recognized_text:
        # Extrage textul recunoscut pe linii
        result = [line["text"] for line in recognized_text]

        # Afișează textul recunoscut
        print("Text recunoscut:")
        for line in result:
            print(line)

        # Calcul metrici
        cer_score = cer(ground_truth, result)
        wer_score = wer(ground_truth, result)
        
        print(f"\nCER (Character Error Rate): {cer_score:.2%}")
        print(f"WER (Word Error Rate): {wer_score:.2%}")
        print()

Text: ['Google Cloud', 'Platform']
Text recunoscut:
Google Cloud
Platform

CER (Character Error Rate): 0.00%
WER (Word Error Rate): 0.00%

Text: ['Succes in rezolvarea', 'TEMELOR la', 'LABORATOARE de', 'Inteligenta Artificiala!']
Text recunoscut:
Lucces in resolvarea
TEMELOR la
LABORA toarele de
Inteligenta Artificialà!

CER (Character Error Rate): 16.18%
WER (Word Error Rate): 122.22%

Text: ['Teodor', 'canta']
Text recunoscut:
Teodor
cânta

CER (Character Error Rate): 9.09%
WER (Word Error Rate): 50.00%

Text: ['Hai sa ne', 'intalnim!']
Text recunoscut:
Hai sã ne
intalmim/

CER (Character Error Rate): 16.67%
WER (Word Error Rate): 75.00%

Text: ['Acesta este', 'un scris urat']
